# Scrape leslibraires.ca for book data

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
# function to extract data from a book's page on the website
def get_book_info(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    data = {'title': soup.find('h1', {'class': 'book-title'}).text,
            'author': soup.find('a', {'class': 'book-meta-author-name'}).text.replace('\n', ''),
            'ISBN':  soup.find('div', {'id': 'sel-buy-box'}).find('span', {'class': 'buy-box--isbn'}).text[6:],
            'summary': soup.find('div', {'class': 'book-summary'}).text.replace('\n', ''),
            'image': soup.find('img', {'class': 'book-image'})['src']}
    return data

In [ ]:
# get data about most popular books on the website
full_data = []
for page in range(1000):
    print(page)
    page = requests.get('https://www.leslibraires.ca/categorie/litterature-LIT/?tri=plus-populaires&ic=25&p={}'.format(page))
    soup = BeautifulSoup(page.content, 'html.parser')
    for book in soup.find_all('a', {'class': 'book-title'}):
        base_url = 'https://www.leslibraires.ca'
        url = base_url + book['href']
        try:
            full_data.append(get_book_info(url))
        except Exception as e:
            print(e)

In [10]:
len(full_data)

16523

In [21]:
full_data[0]

{'title': 'Kukum',
 'author': 'Michel Jean',
 'ISBN': '9782764813447',
 'summary': "Ce roman retrace le parcours d'Almanda Siméon, une orpheline qui va partager sa vie avec les Innus de Pekuakami. Amoureuse d'un jeune Innu, elle réussira à se faire accepter. Elle apprendra l'existence nomade et la langue, et brisera les barrières imposées aux femmes autochtones. Almanda et sa famille seront confrontées à la perte de leurs terres et subiront l'enfermement des réserves et la violence des pensionnats. Racontée sur un ton intimiste, l'histoire de cette femme, qui se déroule sur un siècle, exprime l'attachement aux valeurs ancestrales des Innus et au besoin de liberté qu'éprouvent les peuples nomades, encore aujourd'hui.",
 'image': '//images.leslibraires.ca/books/9782764813447/front/9782764813447_medium.jpg',
 'vector': <tf.Tensor: shape=(512,), dtype=float32, numpy=
 array([ 5.36120348e-02,  2.49565076e-02, -3.81482043e-03, -1.70904794e-03,
        -5.69874309e-02,  3.88389714e-02,  6.887

In [ ]:
# remove books appearing more than once
book_data_no_duplicates = []
titles = []
for book in full_data:
    if book['title'] not in titles:
        titles.append(book['title'])
        book_data_no_duplicates.append(book)
        
full_date = book_date_no_duplicates

# Encode description of books using tensorflow

In [ ]:
import tensorflow_hub as hub
import tensorflow_text

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
# add vectors to the data
for element in full_data:
    element['vector'] = embed(element['summary'])[0]

# Make nearest neighbor models

In [7]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [8]:
vectors = [item['vector'] for item in full_data]
X = np.array(vectors)

In [9]:
# calculate similarity based on Euclidean distance
sim = euclidean_distances(X)
indices = np.vstack([np.argsort(-arr) for arr in sim])

In [10]:
# calculate similarity based on cosine distance
cos_sim = cosine_similarity(X)
cos_indices = np.vstack([np.argsort(-arr) for arr in cos_sim])

In [11]:
# find most similar books for each case
for i, book in enumerate(full_data):
    book['euclidean'] = indices[i][1:21]
    book['cosine'] = cos_indices[i][1:21]

In [14]:
# remove vectors from dict
for book in full_data:
    book.pop('vector')

In [18]:
full_data[0]

{'title': 'Kukum',
 'author': 'Michel Jean',
 'ISBN': '9782764813447',
 'summary': "Ce roman retrace le parcours d'Almanda Siméon, une orpheline qui va partager sa vie avec les Innus de Pekuakami. Amoureuse d'un jeune Innu, elle réussira à se faire accepter. Elle apprendra l'existence nomade et la langue, et brisera les barrières imposées aux femmes autochtones. Almanda et sa famille seront confrontées à la perte de leurs terres et subiront l'enfermement des réserves et la violence des pensionnats. Racontée sur un ton intimiste, l'histoire de cette femme, qui se déroule sur un siècle, exprime l'attachement aux valeurs ancestrales des Innus et au besoin de liberté qu'éprouvent les peuples nomades, encore aujourd'hui.",
 'image': '//images.leslibraires.ca/books/9782764813447/front/9782764813447_medium.jpg',
 'euclidean': array([ 5317, 13539,  8225, 10370,  5505,  6833,  4744, 12761,  7117,
         3384,  1248, 10475,  8596,  2786, 13926, 12419,  3677, 10752,
         7145, 10306], dtype

In [16]:
# save the data
import pickle

with open('books.pkl', 'wb') as f:
    pickle.dump(full_data, f)